# List of Exposures with Hologram


- work with Weakly_2022_34
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/bin/python3


In [2]:
print(sys.path)

['/home/d/dagoret/notebooks/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2022_34/atmospec/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/spectractor/g90263d33d7/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_lsst/g61e05cf539+a8371b4bcb/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/pipe_tasks/gf10b05e212+a1470f6de0/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/shapelet/gd877ba84e5+2f58e93c51/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/meas_modelfit/gf819d2c871+5816c8c9ab/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/meas_extensions_shapeHSM/g6072bfb783+0a325c68f9/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/scarlet_extensions/g9d18589735+cc492336a9/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/proxmin/g33

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2022_34/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/eups 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2022_34
   g0b29ad24fb+897c4bc163 	current w_2022_34 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [6]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
import lsst.daf.butler as dafButler

In [8]:
repo = "/sdf/group/rubin/repo/main"
butler = dafButler.Butler(repo)
registry = butler.registry

## List of Exposures

In [9]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle'])

In [10]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [11]:
df_exposure

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
0,2022062800004,AT_O_20220628_000004,20220628,4,bias,Park position,SDSSr~empty,NaN,0.0,NaN,NaN,NaN
1,2022070600001,AT_O_20220706_000001,20220706,1,test,UNKNOWN,unknown~unknown,NaN,5.0,NaN,NaN,NaN
2,2022070700001,AT_O_20220707_000001,20220707,1,bias,UNKNOWN,empty~ronchi170lpmm,NaN,0.0,NaN,NaN,NaN
3,2022071100001,AT_O_20220711_000001,20220711,1,bias,UNKNOWN,FELH0600~holo4_003,NaN,0.0,NaN,NaN,NaN
4,2022071200001,AT_O_20220712_000001,20220712,1,bias,UNKNOWN,SDSSg~holo4_003,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
44232,2022060900935,AT_O_20220609_000935,20220609,935,cwfs,HD 221054,SDSSg~empty,21.707667,30.0,352.285362,-51.019608,269.453648
44233,2022060900936,AT_O_20220609_000936,20220609,936,cwfs,HD 221054,SDSSg~empty,21.670582,30.0,352.28535,-51.019575,269.453634
44234,2022060900937,AT_O_20220609_000937,20220609,937,cwfs,HD 221054,SDSSg~empty,21.594164,30.0,352.285373,-51.019592,269.453635
44235,2022060900938,AT_O_20220609_000938,20220609,938,cwfs,HD 221054,SDSSg~empty,21.559853,30.0,352.285369,-51.019605,269.453759


## Selection of science

In [12]:
df_science = df_exposure[df_exposure.type == 'science']

In [13]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_26717/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [14]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
2065,2022063000741,AT_O_20220630_000741,20220630,741,science,AUXTEL_DRP_IMAGING:SDSS-J203722_005,SDSSr~empty,36.216657,30.0,309.341775,-5.594321,50.470185
2066,2022063000742,AT_O_20220630_000742,20220630,742,science,AUXTEL_DRP_IMAGING:SDSS-J203722_005,SDSSr~empty,36.311557,30.0,309.341675,-5.594353,50.472142
2067,2022063000743,AT_O_20220630_000743,20220630,743,science,AUXTEL_DRP_IMAGING:SDSS-J203722_006,SDSSr~empty,36.36748,30.0,309.436196,-5.594359,50.230394
2068,2022063000744,AT_O_20220630_000744,20220630,744,science,AUXTEL_DRP_IMAGING:SDSS-J203722_006,SDSSr~empty,36.459853,30.0,309.436102,-5.594346,50.231639
2069,2022063000745,AT_O_20220630_000745,20220630,745,science,AUXTEL_DRP_IMAGING:A2390_016,SDSSr~empty,48.571891,30.0,328.698147,17.412737,83.126668
2070,2022063000746,AT_O_20220630_000746,20220630,746,science,AUXTEL_DRP_IMAGING:A2390_016,SDSSr~empty,48.597445,30.0,328.698026,17.412692,83.124157
2071,2022063000747,AT_O_20220630_000747,20220630,747,science,AUXTEL_DRP_IMAGING:A2390_022,SDSSr~empty,48.759167,30.0,328.50127,17.506591,82.501275
2072,2022063000748,AT_O_20220630_000748,20220630,748,science,AUXTEL_DRP_IMAGING:A2390_022,SDSSr~empty,48.786121,30.0,328.501103,17.506714,82.502325
2073,2022063000749,AT_O_20220630_000749,20220630,749,science,AUXTEL_DRP_IMAGING:A2390_023,SDSSr~empty,48.801233,30.0,328.599795,17.506644,82.299976
2074,2022063000750,AT_O_20220630_000750,20220630,750,science,AUXTEL_DRP_IMAGING:A2390_023,SDSSr~empty,48.82869,30.0,328.599615,17.506728,82.298997


In [15]:
df_science.day_obs.unique()

array([20200127, 20200128, 20200129, 20200212, 20200217, 20200218,
       20200219, 20200220, 20200221, 20200312, 20200313, 20200314,
       20200315, 20200316, 20210119, 20210120, 20210121, 20210126,
       20210209, 20210216, 20210217, 20210218, 20210309, 20210310,
       20210311, 20210323, 20210524, 20210525, 20210608, 20210609,
       20210610, 20210706, 20210707, 20210708, 20210727, 20210804,
       20210817, 20210907, 20210908, 20210909, 20211005, 20211006,
       20211102, 20211103, 20211104, 20211130, 20220208, 20220209,
       20220215, 20220216, 20220217, 20220308, 20220311, 20220316,
       20220317, 20220405, 20220406, 20220407, 20220503, 20220504,
       20220505, 20220524, 20220525, 20220526, 20220607, 20220608,
       20220609, 20220628, 20220629, 20220630])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [16]:
date_sel = 20220630

In [17]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [18]:
len(df_science_selected)

558

# List of filters

In [19]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~holo4_003', 'BG40~holo4_003', 'FELH0600~holo4_003',
       'empty~ronchi170lpmm', 'BG40~ronchi170lpmm',
       'FELH0600~ronchi170lpmm', 'SDSSr~empty'], dtype=object)

In [20]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003', 'BG40~holo4_003', 'FELH0600~holo4_003'],
      dtype='<U18')

## Selection of filter

- But now we have a loop on filters

In [21]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [22]:
pd.set_option('display.max_rows', None)

In [23]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20220630_filt_empty-holo4_003.list
BG40~holo4_003
visitdispersers_20220630_filt_BG40-holo4_003.list
FELH0600~holo4_003
visitdispersers_20220630_filt_FELH0600-holo4_003.list


/tmp/ipykernel_26717/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_26717/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_26717/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [24]:
len(all_def_science_selected_final)

3

In [25]:
all_def_science_selected_final[0]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
1368,2022063000045,AT_O_20220630_000045,20220630,45,science,spec:HD142331,empty~holo4_003,44.7819,30.0,238.597624,-8.619684,145.262022
1369,2022063000046,AT_O_20220630_000046,20220630,46,science,spec:HD142331,empty~holo4_003,44.635934,30.0,238.597606,-8.619646,145.261566
1383,2022063000060,AT_O_20220630_000060,20220630,60,science,spec:HD142331,empty~holo4_003,42.779127,30.0,238.599505,-8.620962,144.104081
1384,2022063000061,AT_O_20220630_000061,20220630,61,science,spec:HD142331,empty~holo4_003,42.635099,30.0,238.599486,-8.62097,144.104102
1399,2022063000076,AT_O_20220630_000076,20220630,76,science,spec:HD185975,empty~holo4_003,60.4983,30.0,307.139255,-87.516853,162.963716
1400,2022063000077,AT_O_20220630_000077,20220630,77,science,spec:HD185975,empty~holo4_003,60.490649,30.0,307.1377,-87.516873,162.963109
1414,2022063000091,AT_O_20220630_000091,20220630,91,science,spec:HD142331,empty~holo4_003,38.374764,30.0,238.601248,-8.620652,141.191952
1415,2022063000092,AT_O_20220630_000092,20220630,92,science,spec:HD142331,empty~holo4_003,38.236606,30.0,238.601094,-8.62034,141.191566
1429,2022063000106,AT_O_20220630_000106,20220630,106,science,spec:HD142331,empty~holo4_003,36.490526,30.0,238.602593,-8.619824,139.344987
1430,2022063000107,AT_O_20220630_000107,20220630,107,science,spec:HD142331,empty~holo4_003,36.355539,30.0,238.602517,-8.619824,139.345002


In [26]:
all_def_science_selected_final[1]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
1370,2022063000047,AT_O_20220630_000047,20220630,47,science,spec:HD142331,BG40~holo4_003,44.476529,30.0,238.597621,-8.61967,145.261569
1371,2022063000048,AT_O_20220630_000048,20220630,48,science,spec:HD142331,BG40~holo4_003,44.334212,30.0,238.597599,-8.619655,145.261571
1385,2022063000062,AT_O_20220630_000062,20220630,62,science,spec:HD142331,BG40~holo4_003,42.484563,30.0,238.599494,-8.620975,144.104278
1386,2022063000063,AT_O_20220630_000063,20220630,63,science,spec:HD142331,BG40~holo4_003,42.340928,30.0,238.599514,-8.62091,144.104169
1401,2022063000078,AT_O_20220630_000078,20220630,78,science,spec:HD185975,BG40~holo4_003,60.482509,30.0,307.137198,-87.516871,162.963303
1402,2022063000079,AT_O_20220630_000079,20220630,79,science,spec:HD185975,BG40~holo4_003,60.474734,30.0,307.137214,-87.516875,162.964278
1416,2022063000093,AT_O_20220630_000093,20220630,93,science,spec:HD142331,BG40~holo4_003,38.088915,30.0,238.601048,-8.620379,141.191767
1417,2022063000094,AT_O_20220630_000094,20220630,94,science,spec:HD142331,BG40~holo4_003,37.9511,30.0,238.601032,-8.62043,141.191556
1431,2022063000108,AT_O_20220630_000108,20220630,108,science,spec:HD142331,BG40~holo4_003,36.208452,30.0,238.602622,-8.619797,139.345048
1432,2022063000109,AT_O_20220630_000109,20220630,109,science,spec:HD142331,BG40~holo4_003,36.077178,30.0,238.60257,-8.619762,139.344915


In [27]:
all_def_science_selected_final[2]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
1372,2022063000049,AT_O_20220630_000049,20220630,49,science,spec:HD142331,FELH0600~holo4_003,44.181785,30.0,238.597515,-8.619709,145.261545
1373,2022063000050,AT_O_20220630_000050,20220630,50,science,spec:HD142331,FELH0600~holo4_003,44.036499,30.0,238.597629,-8.619638,145.261701
1387,2022063000064,AT_O_20220630_000064,20220630,64,science,spec:HD142331,FELH0600~holo4_003,42.190713,30.0,238.599506,-8.620921,144.104262
1388,2022063000065,AT_O_20220630_000065,20220630,65,science,spec:HD142331,FELH0600~holo4_003,42.04734,30.0,238.599505,-8.620938,144.104228
1403,2022063000080,AT_O_20220630_000080,20220630,80,science,spec:HD185975,FELH0600~holo4_003,60.466945,30.0,307.136986,-87.516859,162.963418
1404,2022063000081,AT_O_20220630_000081,20220630,81,science,spec:HD185975,FELH0600~holo4_003,60.459332,30.0,307.137184,-87.516884,162.964405
1418,2022063000095,AT_O_20220630_000095,20220630,95,science,spec:HD142331,FELH0600~holo4_003,37.810374,30.0,238.600995,-8.620363,141.191433
1419,2022063000096,AT_O_20220630_000096,20220630,96,science,spec:HD142331,FELH0600~holo4_003,37.67315,30.0,238.601062,-8.620312,141.191671
1433,2022063000110,AT_O_20220630_000110,20220630,110,science,spec:HD142331,FELH0600~holo4_003,35.936789,30.0,238.602577,-8.619822,139.344998
1434,2022063000111,AT_O_20220630_000111,20220630,111,science,spec:HD142331,FELH0600~holo4_003,35.803062,30.0,238.602655,-8.619746,139.344909


In [28]:
all_def_science_selected_final[3]

IndexError: list index out of range

In [ ]:
all_def_science_selected_final[4]